In [ ]:
# stanza.download('fr')

In [ ]:
import scraper
## save models, data...
import pickle
## tokeniseur
import stanza
import spacy 
## stopwords
from nltk.corpus import stopwords
import chiffres # stopwords définis par nous
import string
## for data
import pandas as pd
import numpy as np
## for progress
from tqdm import tqdm
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
# import transformers

In [ ]:
nlp_spacy = spacy.load('fr_core_news_md',disable = ['ner','parser'])

In [ ]:
# stopwords de nltk
nltkStopWords = set(stopwords.words('french'))
# nlp_spacy = spacy.load('fr_core_news_md')
# des poncuations du package string avec «, »
myPonct = { ponct for ponct in string.punctuation} | {"«","»"}
# le tout couplé avec les stopwords de spacy
myStopwords = nltkStopWords | chiffres.customize_stopwords | nlp_spacy.Defaults.stop_words\
       | myPonct
print(f'Les mots vides finaux sont au nombre de {len(myStopwords)}.')

In [ ]:
# lire le corpus
df = pd.read_csv('corpus_1000.csv')
# filtrer les categories
# df = df[df.label.isin(['culture','sport','economie'])]
# 1000 articles par categorie
df.shape

In [ ]:
# 5 articles aleatoires
df.sample(5) 

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("nombre d'articles par catégorie", fontsize=12)
df["label"].reset_index().groupby("label").count().sort_values(by= 
       "index").plot(kind="barh", legend=False, 
        ax=ax).grid(axis='x')
plt.show()

In [ ]:
doc_spacy = nlp_spacy("Emmanuel Macron est un bon politique qui vit à l'Avenue des Champs-Élysées.")

In [ ]:
# test spacy, mot tiret séparé
for word in doc_spacy:
    print(word.text,end="|")

In [ ]:
nlp_stanza = stanza.Pipeline(lang='fr',processors='tokenize,mwt,lemma,ner')

In [ ]:
doc_test = nlp_stanza("Emmanuel Macron est une bon politique qui vit à l'Avenue des Champs-Élysées. Jean de La Fontaine est un écrivain français.")
for token in doc_test.sentences[0].tokens:
    print(token.text,end="|")

In [ ]:
# entree : sentence object of stanza
def tokenizer(text, flag_lemm=True, flag_lowercase=True, flag_rm_stop=True):
    # run stanza
    stanza_obj = nlp_stanza(text)
    final_sentences = []
    # keep the named entities together
    for sent in stanza_obj.sentences:
        debut = []
        final_tokens = []
        len_sent = len(sent.tokens) -1
        for i,token in enumerate(sent.tokens):
            if token.ner != "O":
                debut.append(token.text)
                if i == len_sent:
                    final_tokens.append(" ".join(debut))
            else:
                if debut:
                    final_tokens.append(" ".join(debut))
                if flag_lemm: 
                    final_tokens.append(token.words[0].lemma)
                else:
                    final_tokens.append(token.words[0].text)
                debut = []
        # lowercase
        if flag_lowercase:
            final_tokens = [w.lower() for w in final_tokens]
        # remove stopwords
        if flag_rm_stop:
            final_tokens = [w for w in final_tokens if w not in myStopwords]
        ## back to string from list
        final_text = "|".join(final_tokens)
        # print(final_text)
        final_sentences.append(final_text)
        final_tokens = []
    return "||".join(final_sentences)

In [ ]:
test_phrase = "Emmanuel Macron est une bon politique qui vit à l'Avenue des Champs-Élysées. Jean de La Fontaine est un écrivain français."
test_allflags = tokenizer(test_phrase)
test_withstop = tokenizer(test_phrase,flag_rm_stop=False)
test_allflags_nolemma = tokenizer(test_phrase,flag_lemm=False)

In [ ]:
test_allflags

In [ ]:
print(test_allflags)
print(test_withstop)
print(test_allflags_nolemma)

In [ ]:
df["text_clean"] = df["text"].apply(lambda x: 
          tokenizer(x))

In [ ]:
len(df["text_clean"])

In [ ]:
# save to pickle
with open(r"corpus1000_5themes_cleaned.pickle", "wb") as output_file:
    pickle.dump(df, output_file)

In [ ]:
df_complete = scraper.decompress_pickle("corpus_complet_5themes.pbz2")

In [ ]:
# type(df_complete.label)

In [80]:
# get a sample of 9000 articles for eco politique et societe
df_sampled = df_complete[df_complete.label.isin(['politique','societe','economie'])]
df_sampled = df_complete.groupby('label')['text'].apply(lambda s: s.sample(9000))

In [81]:
df_sampled = df_sampled.tolist()

In [77]:
liste_des_phrases = []
for text in df_sampled:
    liste_des_phrases.append(tokenizer(text))